# Opis:
Zadanie czwarte polegało na rozwiązaniu tzw. Scheduling Problem, czyli problemu polegający na przypisaniu pracowników każdego dnia, tak aby zminimalizować ich koszty zatrudnienia.
![title](./4.png)

In [5]:
from pulp import *
import pandas as pd

pracownicy = ["Ania","Stefan","Hektor","Olaf","Lidia","Piotr"]
cena_h = [100,50,60,40,110,70] # Stawka godzinowa każdego pracownika.
dni = ["Pon","Wt","Sr","Czw","Pia"]
ilosc_pracownikow_dnia = [2,1,1,1,3] # Ile jednego dnia może być pracowników.
sum_tab = [] # lista przeznaczona do sumowania elementów.
pracownicy_dzien = [] # Lista przeznaczona do przechowywania list zawierajacych zmienna binarna odpowiedzialna za dostepnosc pracownika kazdego dnia np. AniaPon, AniaWt itd.
kiedy_nie_moze = [[],[1,2,3,4],[3,4],[],[0,1,2],[3,4]] # Lista, zawieraja liste, które mówią w który dzień (przedstawiony liczbowo 0-pon 1-wt itd.) dany pracownik nie moze pracować, listy ułozone tak jak pracownicy.

prob = LpProblem("Sklep", LpMinimize)

for pracownik in pracownicy: #For wypełniający wyżej zdefiniowaną tablice pracownicy_dzien zmiennymi decyzyjnymi AniaPon itd.
    pracownik = [LpVariable(pracownik+"{}".format(dzien), cat = "Binary") for dzien in dni] 
    pracownicy_dzien.append(pracownik)
 
prob += sum((8*(cena*(sum(x))) for x,cena in zip(pracownicy_dzien,cena_h))) #Obliczanie sumy kosztów zatrudnienia pracowników

for numer,pracownik in enumerate(pracownicy_dzien): #For służący do utworzenia limitów dla poszczególnych praconików, w jakie dni mogą pracować
    for nie_moze in kiedy_nie_moze[numer]:
        sum_tab.append(pracownik[nie_moze])
    prob += sum(sum_tab) == 0 
    sum_tab.clear()

for dzien in range(len(dni)): #For służący do utworzenia limitów ile praconików może być jednego dnia w pracy.
    for pracownik in range(len(pracownicy)):
        sum_tab.append(pracownicy_dzien[pracownik][dzien])
    prob += sum(sum_tab) == ilosc_pracownikow_dnia[dzien]
    sum_tab.clear()

for pracownik in pracownicy_dzien: #For służący do utrworzenia limitu ile dni w tygodniu może pracować dany pracownik
    prob += sum(pracownik) <= 3

prob.writeLP("Scheduling_Problem")
prob.solve()
print ("Status:", LpStatus[prob.status])
for v in prob.variables():
    if(v.varValue == 1):
        print (v.name, "=", v.varValue)
print("Minimalny koszt = ", value(prob.objective))

Sklep:
MINIMIZE
None
VARIABLES

Status: Optimal
AniaPia = 1.0
HektorPon = 1.0
HektorSr = 1.0
LidiaPia = 1.0
OlafCzw = 1.0
OlafPia = 1.0
OlafWt = 1.0
StefanPon = 1.0
Minimalny koszt =  4000.0
